# Mikrokosmos: a lambda calculus tutorial

This tutorial should cover

 - Basic lambda calculus.
 - Pairs and logic.
 - Church numerals and arithmetic.
 - SKI combinators.
 - Data structures: pairs, lists and trees.
 - Recursion.

## 1. Basic lambda calculus

In [38]:
id = \x.x
const = \x.\y.x
compose = \f.\g.\x.f (g x)

## 2. Logic

### 2.1. The booleans

Boolean logic can be encoded in lambda calculus. Our intuition on what means to be a truth value is that it can distinghish between two values (**true** or **false**) or two branches on a program (if ... else ...).

We are going to use this intuition to write an encoding of boolean values based on their ability to choose between two branches. Maybe surprisingly, this encoding will be also useful to write the usual boolean logic gates.

In [2]:
# Church encoding of boolean truth values
true  = \a.\b.a
false = \a.\b.b

Here, a truth value is a function on two elements that chooses one of them.

 - $\mathtt{true}\ a\ b  = a$
 - $\mathtt{false}\ a\ b = b$

This is called the *Church encoding* of the booleans, as it was used by Alonzo Church. This idea of defining a type based not on its content but on how it can be used will appear later, when we define more complex data structures. 

In [3]:
# Examples
true id const
false id const
true true false
false true false

λa.a ⇒ id
λa.λb.a ⇒ true, const
λa.λb.a ⇒ true, const
λa.λb.b ⇒ false


In particular, `true` is exactly the same lambda term as `const`.

### 2.2. If-else 

The advantage of this way of encoding the boolean values is that they can be easily used in combination with other lambda terms. In particular, the way to encode an if-else is almost trivial: it is already encoded on the lambda terms!

In [4]:
# If true, then the id function will be returned
# if false, then the const function will be returned 
(\b. b id const) true
(\b. b id const) false

λa.a ⇒ id
λa.λb.a ⇒ true, const


If we really want to write an `if-else` function, it will be, quite literally, a trivial one

In [5]:
ifelse = \b.b
(ifelse true) id const
(ifelse false) id const

λa.a ⇒ ifelse, id
λa.λb.a ⇒ true, const


### 2.3. Logic gates

Usual operations on booleans can be defined too on this encoding and they will be surprisingly easy if we think of booleans as functions choosing from two terms.

In [6]:
# The and gate takes two booleans and returns a true if and only if 
# the two given booleans are true. 
and = \p.\q.p q p

In [7]:
# Checking the truth table for the and gate
and true true
and true false
and false true
and false false

λa.λb.a ⇒ true, const
λa.λb.b ⇒ false
λa.λb.b ⇒ false
λa.λb.b ⇒ false


**Exercise:** Think why this definition of the `and` gate works.

*Hint: think what happens when the first argument is a `true`. What happens if it is a `false`?*

The `or` gate can be defined in a similar way.

In [8]:
# The or gate takes two booleans and returns a true if and only if
# any of them (or both) are true.
or = \p.\q.p p q

In [9]:
# Checking the truth table for the and gate
or true true
or true false
or false true
or false false

λa.λb.a ⇒ true, const
λa.λb.a ⇒ true, const
λa.λb.a ⇒ true, const
λa.λb.b ⇒ false


And finally, the negation operator is only a way of interchanging the two truth values

In [10]:
not = \b.b false true

In [16]:
not true
not false
not (and true true)

λa.λb.b ⇒ false
λa.λb.a ⇒ true, const
λa.λb.b ⇒ false


The boolean logic implication operator works also as a boolean gate, it can be defined as

$$(a \to b) \equiv (\neg a) \vee b,$$

that is, the implication is true if both are true or if the premise is false.

In [21]:
implies = \a.\b.or (not a) b

**Exercise:** Compute the logic table for the implication using the previous definition.

In [ ]:
# -- Your solution goes here

**Exercise:** Compute the following logic clauses using lambda calculus
 
 - True or false implies false.
 - False implies that: false implies false.
 - The negation of false and the negation of true both imply true.

In [ ]:
# -- Your solution goes here

**Exercise:** Define the `xor` gate as a lambda term. The `xor` of two boolean values must return a true if and only if *exactly one* of them are true. Check also its logic table.

*Hint: you may want to use the already defined `not`.*

In [14]:
# -- Your solution goes here

## 3. Church numerals and arithmetic

### 3.1. Peano and the natural numbers

In the 19th century, Giuseppe Peano gave a definition of the natural numbers and an axiomatic theory of them based on only two contructors

 - The zero is a natural number, written as Z.
 - The successor of a natural number is a natural number, written as S.
 
In those terms, the usual natural numbers will be 

$$ Z,\ SZ,\ S(SZ),\ S(S(SZ)),\ \dots $$
 
The question is now how can we encode them on lambda calculus. We do not have the ability to write the two constructors on lambda calculus, so we will make the natural numbers depend on them. This is again the same idea we used when we tried to encode booleans, we do not care about the content, but about how can we use them later.

In [24]:
# Definition of the natural numbers
0 = \s.\z.z
succ = \n.\s.\z.s (n s z)

This definition of `0` is trivial: given a successor function and a zero, return the zero. The successor function seems more complex, but it uses the same underlying idea: given a number, a successor and a zero, apply the successor to the interpretation of that number using the same successor and zero.

In [26]:
# Names of the first twenty natural numbers
1  = succ 0
2  = succ 1
3  = succ 2
4  = succ 3
5  = succ 4
6  = succ 5
7  = succ 6
8  = succ 7
9  = succ 8
10 = succ 9
11 = succ 10
12 = succ 11
13 = succ 12
14 = succ 13
15 = succ 14
16 = succ 15
17 = succ 16
18 = succ 17
19 = succ 18
20 = succ 19

Under this interpretation, a number `n` is really a function taking a function `a` as an argument and applying it `n` times over the argument `b`.

In [32]:
5

λa.λb.(a (a (a (a (a b))))) ⇒ 5


In [33]:
5 not true
4 not false

λa.λb.b ⇒ 0, false
λa.λb.b ⇒ 0, false


**Exercise:** Define a function that takes a natural number and returns true if and only if the number is even.

*Hint: you may want to interpret the given number as a function.*

In [ ]:
# -- Your solution goes here

### 3.2. Addition and multiplication

The encoding of the addition and multiplication of natural numbers will profit from the interpretation of numbers as functions. This is, in fact, the only way we can use naturals; but we will quickly see that this is an strenght instead of a weakness of our encoding. We are really encoding naturals as their induction principle: we can define a function by defining a zero and a successor.

The `double` function will only change the successor for the composition of the successor function with itself.

In [39]:
double = \n.\s.\z.n (compose s s) z

In [43]:
double 0
double 3
double 4

λa.λb.b ⇒ 0, false
λa.λb.(a (a (a (a (a (a b)))))) ⇒ 6
λa.λb.(a (a (a (a (a (a (a (a b)))))))) ⇒ 8


**Exercise:** Define a `triple` function.

In [ ]:
# -- Your solution goes here

We are going now to define **addition** using this same principle. It takes a successor and a zero, computes the first number as `(n s z)` and then uses it as a zero on the interpretation of the second one.

In [46]:
plus = \m.\n.\s.\z.m s (n s z)

In [47]:
plus 2 1
plus 3 4
plus 0 5

λa.λb.(a (a (a b))) ⇒ 3
λa.λb.(a (a (a (a (a (a (a b))))))) ⇒ 7
λa.λb.(a (a (a (a (a b))))) ⇒ 5


**Exercise:** How would you define multiplication? Keep in mind that you can use a number as a function. Keep also in mind the previous exercises on `double` and `triple`.

*Spoilers below!*

In [50]:
# -- Your solution goes here
# mymult =

There many possible ways of defining multiplication. Some of them can use the repeated application of `plus` to a number; but we are going to define **multiplication** in a way that is similar to how we defined `double` previously. We are going to interpret the successor as the n-fold application of successor.

In [52]:
mult = \m.\n.\s.\z.m (n s) z

In [54]:
mult 0 3
mult 1 5
mult 3 4

λa.λb.b ⇒ 0, false
λa.λb.(a (a (a (a (a b))))) ⇒ 5
λa.λb.(a (a (a (a (a (a (a (a (a (a (a (a b)))))))))))) ⇒ 12


### 3.3. The predecessor function

But, how to compute the predecessor of a number? We have not encoded negative numbers, so it could be a function returning zero whenever it tries to get the predecessor of zero. It is an insightful exercise to try to define it by yourself, but please, do not get too obsessed with it. The solution is certainly not easy. 

In [ ]:
# -- You can try here
# -- Spoilers below!

The solution follows this paragraph, but you probably expected something easier! Kleene, who was a student of Alonzo Church, discovered for the first time how to write a predecessor on lambda calculus while at the dentist. This discovery made Church start thinking that every intuitively computable function could be computed using lambda calculus, that is, that the notions of lambda-computable function and intuitively computable function would coincide.

In [56]:
pred = \n.\f.\x.n (\g.(\h.h (g f))) (\u.x) (\u.u)

In [59]:
pred 4
pred 1
pred 0

λa.λb.(a (a (a b))) ⇒ 3
λa.λb.b ⇒ 0, false
λa.λb.b ⇒ 0, false


But why does something like this even work? We will develop an intuition on this kind of constructions later.

**Exercise:** Use the predecessor function to define the `minus` function. It should return the difference between two numbers. If the first number is smaller than the second, it simply should return zero.

In [60]:
# -- Your solution goes here

### 3.3. Predicates on natural numbers (WIP)

## 4. Combinatory logic (WIP)

## 5. Data structures (WIP)

## 6. Recursion